# Web Browsing

###  **Full Browsing**

In [4]:
from aiohttp import ClientSession
from lxml import html
from langchain.schema import HumanMessage, SystemMessage
from fastapi.concurrency import run_in_threadpool

from app.shared import Shared
from app.common.constants import QueryTemplates
from app.utils.logger import CustomLogger, LoggingConfig

logger = CustomLogger(
    "Browsing", logging_config=LoggingConfig(file_log_name="./logs/notebook.log")
)

shared = Shared()
query = "Explain to me what the vision pro is that Apple released."

In [5]:
web_search_llm_output = (
    await shared.web_search_llm.agenerate(
        messages=[[HumanMessage(content=query)]],
    )
).llm_output
assert web_search_llm_output is not None, "Cannot browse the web without a query."
query_to_search = web_search_llm_output["function_calls"][0]["arguments"][
    "query_to_search"
]
logger.info(f"query_to_search: {query_to_search}")

snippets_with_link: dict[str, str] = await run_in_threadpool(
    shared.duckduckgo.formatted_results_with_link, query=query_to_search
)

print("snippets_with_link:")
print(snippets_with_link)

[2023-06-14 19:30:05,177] Browsing:INFO - query_to_search: Apple Vision Pro


snippets_with_link:
{'https://www.apple.com/newsroom/2023/06/introducing-apple-vision-pro/': "# [https://www.apple.com/newsroom/2023/06/introducing-apple-vision-pro/]\n```Introducing Apple Vision Pro: Apple's first spatial computer\nCUPERTINO, CALIFORNIA Apple today unveiled Apple Vision Pro, a revolutionary spatial computer that seamlessly blends digital content with the physical world, while allowing users to stay present and connected to others. Vision Pro creates an infinite canvas for apps that scales beyond the boundaries of a traditional display and introduces a ...\n```", 'https://www.tomsguide.com/reviews/apple-vision-pro': '# [https://www.tomsguide.com/reviews/apple-vision-pro]\n```Apple Vision Pro hands-on: A mixed reality breakthrough\nThe Apple Vision Pro is a very powerful mixed reality headset with a sleek design, eye and hand tracking and sharp micro-OLED displays. This device offers immersive video watching, serious...\n```', 'https://techcrunch.com/2023/06/05/apple-he

In [6]:
async def scrolling(
    link: str,
    tokens_per_chunk: int,
    chunk_overlap: int,
) -> str | None:
    async with ClientSession() as session:
        res = await session.get(link)
        paragraphs = html.fromstring(await res.text()).xpath("//p")
        scrollable_contents: list[str] = Shared().token_text_splitter.split_text(
            "\n".join([p.text_content().strip() for p in paragraphs]),
            tokens_per_chunk=tokens_per_chunk,
            chunk_overlap=chunk_overlap,
        )
        for scrollable_content in scrollable_contents:
            scrollable_content = scrollable_content.strip()
            logger.info(f"Reading content: {scrollable_content}")
            answerable_or_not_llm_output = (
                await shared.control_web_page_llm.agenerate(
                    messages=[
                        [
                            SystemMessage(content=scrollable_content),
                            HumanMessage(content=query),
                        ]
                    ],
                )
            ).llm_output
            if answerable_or_not_llm_output is None:
                logger.info("Reading content failed.")
                continue
            answerable = answerable_or_not_llm_output["function_calls"][0]["arguments"][
                "answerable"
            ]
            logger.info(f"answerable: {answerable}")
            if answerable:
                logger.info(
                    "I found a link that is sufficient to answer the user's question."
                )
                return scrollable_content
            else:
                continue
        logger.info("This link is not sufficient to answer the user's question.")
        return None

In [12]:
tokens_per_chunk = 1024
chunk_overlap = 256
browsing_result: str | None = None

snippets = "\n\n".join(snippets_with_link.values())
while snippets_with_link:
    browsing_llm_output = (
        await shared.browsing_llm.agenerate(
            messages=[
                [
                    SystemMessage(content=snippets),
                    HumanMessage(content=query),
                ]
            ],
        )
    ).llm_output
    assert browsing_llm_output is not None, "Cannot browse the web anymore."
    arguments = browsing_llm_output["function_calls"][0]["arguments"]
    if arguments["action"] == "click_link":
        logger.info(f"Clicking on link: {arguments['link_to_click']}")
        snippets_with_link.pop(arguments["link_to_click"])
        scroll_result: str | None = await scrolling(
            link=arguments["link_to_click"],
            tokens_per_chunk=tokens_per_chunk,
            chunk_overlap=chunk_overlap,
        )
        if scroll_result is not None:
            logger.info("We've got the answer!")
            browsing_result = scroll_result
            break
        else:
            logger.info("We still don't have the answer.")
            continue
    elif arguments["action"] == "finish_browsing":
        logger.info("Feedback bot decided to finish browsing.")
        browsing_result = snippets
        break
    else:
        logger.info(f"An error occurred while browsing: {arguments['action']}")
        browsing_result = snippets
        break

[2023-06-14 19:46:58,533] Browsing:INFO - Clicking on link: https://www.theverge.com/2023/6/5/23738968/apple-vision-pro-ar-headset-features-specs-price-release-date-wwdc-2023
[2023-06-14 19:47:00,663] Browsing:INFO - Reading content: By  Adi Robertson, a senior tech and policy editor focused on VR, online platforms, and free expression. Adi has covered video games, biohacking, and more for The Verge since 2011.
Apple has announced an augmented reality headset called Apple Vision Pro that “seamlessly” blends the real and digital world. “It’s the first Apple product you look through, and not at,” CEO Tim Cook said of the device, which looks like a pair of ski goggles. As rumored, it features a separate battery pack and is controlled with eyes, hands, and voice. It will start at $3,499 and launch early next year, starting in the US market with more countries coming later in the year.
Vision Pro is positioned as primarily an AR device, but it can switch between augmented and full virtual r

In [14]:
llm_output = await shared.llm.apredict(  # type: ignore
    QueryTemplates.CONTEXT_QUESTION__DEFAULT.format(
        context=browsing_result, question=query
    )
)
print(llm_output)

The Vision Pro is an augmented reality headset released by Apple. It is a wearable device that seamlessly combines the real and digital world. The device, which resembles a pair of ski goggles, features a separate battery pack and can be controlled using eyes, hands, and voice commands. The headset can switch between augmented reality and full virtual reality using a dial.

The Vision Pro operates on an operating system called visionOS, where users can browse rows of app icons by looking at them. It supports Bluetooth accessories such as the Magic Keyboard and Magic Trackpad, and users can connect their Mac to use inside the headset. The device also has downward-facing cameras that can capture hand movements.

The headset has a glass front and an aluminum frame and contains five sensors, 12 cameras, and a 4K display for each eye. It is designed to be adjustable and can fit a variety of face shapes and head sizes. For those who wear glasses, custom optical inserts created by Zeiss can b